# DATATHON FERREYROS

En Ferreyros no solo vendemos máquinas, también nos aseguramos de maximizar la productividad de nuestros clientes y la vida útil de las máquinas, ofreciendo repuestos y servicios de manera proactiva y oportuna.

Por ello, priorizar las oportunidades de venta de repuestos y servicios nos permite brindar el mejor servicio a los clientes que más lo necesitan.

Tu misión consiste en utilizar la data histórica de oportunidades para estimar la probabilidad de cierre de nuevas oportunidades. Adicionalmente, tendrás acceso a data del parque de máquinas y horómetros de las máquinas.

# Librerías


In [ ]:
# Cargar Librerías

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import gc
import sys
from sklearn.feature_selection import VarianceThreshold
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Definición de funciones

In [ ]:
# Definición de funciones


def display_table_info_and_head(table, table_name):
    print("Información de la tabla:",table_name,"\n")
    print(table.info(),"\n")
    print("Mostrar los primeros 5 registros:","\n")
    print(table.head(5))

def get_items_by_opportunity(opportunity_id):
    items = OpportunityItemPostVenta[OpportunityItemPostVenta['OpportunityID'] == opportunity_id]
    return items
​
def get_closed_won_ppportunity(opportunity_id):
    items = X_train_df[X_train_df['OpportunityID'] == opportunity_id]
    return items
​
def get_customer_id(customer_id):
    items = EquipmentPostVenta[EquipmentPostVenta['CurrentCustomerID'] == customer_id]
    return items
​
def get_serial_number_id(serial_id):
    items = horometros_vl[horometros_vl['SerialNumber'] == serial_id]
    return items
​
def fx_porc_missings(data, only_missings = False):
    df_vars_missings = pd.concat([pd.DataFrame(data.isnull().sum(), columns = ['n_nulos']),
           pd.DataFrame(100*data.isnull().sum()/len(data), columns = ['%Total'])], axis = 1)
    if only_missings:
        return(df_vars_missings[df_vars_missings["n_nulos"]!=0])
    else:
        return(df_vars_missings)

def plot_graph_univariate(df, meta):
    import matplotlib
    matplotlib.rcParams.update({'font.size': 14})
    for i in range(len(meta)):
        plt.figure(figsize=(55,8))
        v=meta.iloc[i].variable
        t=meta.iloc[i].tipo
        if (t.__class__.__name__=="CategoricalDtype"):
            fa=df[v].value_counts()
            fr=fa/len(df[v])
            #Barras
            plt.subplot(1,2,1)
            ax=plt.bar(fa.index,fa,edgecolor='black')
            plt.xticks(fa.index,rotation=90)
            plt.title('Distribution of '+v+' categories')

            total = len(df[v])
            max_height=fa.max()
            for p in ax:
                percentage = f'{100 * p.get_height() / total:.1f}%'
                x = p.get_x() + p.get_width() / 2
                y = p.get_height()
                plt.annotate(percentage, (x, y + 0.007*total), ha='center', va='bottom', fontsize=12, rotation=90)
            plt.ylim(0, max_height * 1.3)
            plt.grid(False)

            # Pie
            plt.subplot(1, 2, 2)
            plt.pie(fr, labels=fr.index, autopct='%1.1f%%', shadow=True, startangle=90)
            plt.legend(fr.index, loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
            plt.title('Pie chart of ' + v)
            plt.grid(False)
        else:
            #Histograma
            plt.subplot(1,2,1)
            plt.hist(df[v].dropna(),bins=100,edgecolor='black')
            plt.title('Histogram of '+v)
            #Boxplot
            plt.subplot(1,2,2)
            plt.boxplot(df[v],vert=False)
            plt.title('Boxplot of '+v)
        plt.tight_layout()
        plt.show()
​
​
def plot_graph_bivariate(df2, meta, y):
    import matplotlib
    matplotlib.rcParams.update({'font.size': 16})
    for i in range(len(meta)):
        plt.figure(figsize=(35, 10))
        v = meta.iloc[i].variable
        t = meta.iloc[i].tipo
        if v == y:
            break
        print(v)
        if t == "category":
            g = df2.groupby([df2[y], v],observed=True).size().unstack(0)
            tf = g[1] / (g[0] + g[1])
            g_sorted = g.reindex(tf.sort_values(ascending=False).index)
            c1 = g_sorted[0]
            c2 = g_sorted[1]
            tf_sorted = tf.sort_values(ascending=False)
            width = 0.9
​
            p1 = plt.bar(g_sorted.index, c1, width)
            p2 = plt.bar(g_sorted.index, c2, width, bottom=c1)
​
            plt.ylabel('Freq')
            plt.title('Bivariate: ' + v)
            plt.xticks(g_sorted.index, rotation=90)
            plt.legend((p1[0], p2[0]), ('0', '1'), loc='lower left', bbox_to_anchor=(1, 1))
​
            tf_line = plt.twinx().plot(tf_sorted.values, linestyle='-', linewidth=3.0, color='black', marker="o")
            plt.ylabel(y)

            for x, val in enumerate(tf_sorted.values):
                plt.text(x, val+0.0003, f'{val:.2f}', color='black', fontsize=12, ha='center', va='bottom',rotation=90)

        else:
            d, bins = pd.qcut(df2[v], 10, retbins=True, duplicates='drop', labels=False)
            g = df2.groupby([y, d]).size().unstack(0)
            tf = g[1] / (g[0] + g[1])
            g_sorted = g.reindex(tf.sort_values(ascending=False).index)
            N = len(g_sorted)
            mMeans = g_sorted[0]
            wMeans = g_sorted[1]
            tf_sorted = tf.sort_values(ascending=False)
            ind = np.arange(N)
​
            width = 0.9
            p1 = plt.bar(ind, mMeans, width)
            p2 = plt.bar(ind, wMeans, width, bottom=mMeans)
​
            plt.ylabel('Freq')
            plt.xlabel("Deciles " + v)
            plt.title('Bivariate: ' + v + " vs " + y)
            bin_labels = [f'{bins[i]:.2f} - {bins[i+1]:.2f}' for i in range(len(bins) - 1)]
            plt.xticks(ind, bin_labels, rotation=90)
            plt.legend((p1[0], p2[0]), ('0', '1'), loc='lower left', bbox_to_anchor=(1, 1))
​
            tf_line = plt.twinx().plot(tf_sorted.values, linestyle='-', linewidth=2.0, color='black')
            plt.ylabel(y)

            for x, val in enumerate(tf_sorted.values):
                plt.text(x, val+0.0003, f'{val:.2f}', color='black', fontsize=12, ha='center', va='bottom',rotation=90)

        plt.show()

​
def create_columns_based_nulls(df,exclude_cols=None,name='field'):
    if exclude_cols is None:
        exclude_cols = []

    # Crear columnas indicadoras de valores faltantes
    missing_cols = df.columns[df.isna().any()].tolist()
    for col in missing_cols:
        if col not in exclude_cols:
            df[f'{col}_missing_{name}'] = df[col].isna().astype(int)

    return df
​
def handle_missing_values_9999(df,fill_value=9999, exclude_cols=None):
    if exclude_cols is None:
        exclude_cols = []

    # Llenar valores faltantes con el valor extremo, excluyendo columnas especificadas
    for col in df.columns:
        if col not in exclude_cols:
            df[col].fillna(fill_value, inplace=True)

    return df
​
def handle_nans_based_on_criteria(df):
    # Rellenar count y sum con 0 cuando NaN
    count_sum_cols = [col for col in df.columns if 'count' in col or 'sum' in col]
    df[count_sum_cols] = df[count_sum_cols].fillna(0)

    # Rellenar std con 0 cuando solo hay un registro, y con 99999 cuando falta la categoría específica
    std_cols = [col for col in df.columns if 'std' in col]
    for col in std_cols:
        df[col] = df.apply(lambda row: 0 if row.get(col.replace('std', 'count'), 0) == 1 else 99999 if pd.isna(row[col]) else row[col], axis=1)

    # Rellenar otros NaNs con 99999
    other_cols = [col for col in df.columns if col not in count_sum_cols + std_cols]
    df[other_cols] = df[other_cols].fillna(99999)

    return df
​
​
def correlation_heatmap(df):
    """Función para plotear las correlaciones de las variables de un dataset"""

    _ , ax = plt.subplots(figsize =(10, 6))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)

    _ = sns.heatmap(
        df.corr(),
        cmap = colormap,
        square=True,
        cbar_kws={'shrink':.9 },
        ax=ax,
        annot=True,
        fmt='.2f',
        linewidths=0.1,
        vmax=1.0,
        vmin=-1.0,
        linecolor='white',
        annot_kws={'fontsize':8}
    )
    plt.title('Pearson Correlation of Features', y=1.05, size=12)
    plt.show()

​
def drop_highly_correlated_features(df, threshold=0.9):
    """
    Analyzes and drops highly correlated numerical features from the dataframe.

    Parameters:
    df (pd.DataFrame): The input dataframe.
    threshold (float): The correlation threshold to identify highly correlated features.

    Returns:
    pd.DataFrame: The dataframe with highly correlated numerical features dropped.
    """
    # Select only numerical features
    numerical_df = df.select_dtypes(include=['int64', 'float64'])

    # Calculate the correlation matrix
    corr_matrix = numerical_df.corr().abs()
​
    # Select the upper triangle of the correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
​
    # Find features with correlation greater than the threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
​
    # Drop the highly correlated features from the original dataframe
    df_dropped = df.drop(columns=to_drop)
​
    print(f"Dropped columns: {to_drop}")
​
    return df_dropped
​
# Example usage:
# df_cleaned = drop_highly_correlated_features(df, threshold=0.9)
​
​
def feature_selection_customized_initial(id_column, train_data, variance_threshold=0.005, max_missing_percentage=0.9, correlation_threshold=0.85):
    """
    Perform feature selection based on variance, missing values, and correlation.
​
    Args:
    - id_column (string): ID of the row
    - train_data (pd.DataFrame): Training data.
    - variance_threshold (float): Threshold for variance-based feature selection.
    - max_missing_percentage (float): Maximum allowed percentage of missing values.
    - correlation_threshold (float): Threshold for correlation-based feature selection.
​
    Returns:
    - train_data (pd.DataFrame): Dataframe after feature selection.
    """
    # 0. Replacing inf values
    train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    cols_to_keep_vt = set(train_data.columns)
    cols_to_keep_na = set(train_data.columns)
    cols_to_keep_corr = set(train_data.columns)

    # 1. Variance Threshold
    if variance_threshold > 0:
        numerical_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
        numerical_cols = [col for col in numerical_cols if col != id_column]
        selector = VarianceThreshold(threshold=variance_threshold)
        selector.fit(train_data[numerical_cols])
        cols_to_keep_vt = set(train_data[numerical_cols].columns[selector.get_support(indices=True)].tolist())
​
    # 2. Missing Values
    if max_missing_percentage > 0:
        missing_percentage = train_data.isnull().mean()
        cols_to_drop_na = missing_percentage[missing_percentage > max_missing_percentage].index
        cols_to_keep_na = set(train_data.columns) - set(cols_to_drop_na)
​
    train_data = train_data[list(cols_to_keep_vt.intersection(cols_to_keep_na))]
​
    # 3. Correlation Matrix Elimination
    if correlation_threshold > 0:
        # Calculate correlation matrix in chunks to avoid memory issues
        chunk_size = 25
        correlated_features = set()
        for i in range(0, len(train_data.columns), chunk_size):
            chunk = train_data.iloc[:, i:i+chunk_size]
            correlation_matrix = chunk.corr().abs()
            upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
            for column in upper_triangle.columns:
                if any(upper_triangle[column] > correlation_threshold):
                    correlated_features.add(column)
                    cols_to_keep_corr = set(train_data.columns) - set(correlated_features)
​
    return cols_to_keep_corr.union(cols_to_keep_na).union(cols_to_keep_vt).union(set([id_column]))
​
​
from sklearn.feature_selection import VarianceThreshold
def feature_selection_customized_last(id_column, train_data, variance_threshold=0.005, max_missing_percentage=0.9, correlation_threshold=0.85):
    """
    Perform feature selection based on variance, missing values, and correlation.

    Args:
    - id_column (string): ID of the row
    - train_data (pd.DataFrame): Training data.
    - variance_threshold (float): Threshold for variance-based feature selection.
    - max_missing_percentage (float): Maximum allowed percentage of missing values.
    - correlation_threshold (float): Threshold for correlation-based feature selection.

    Returns:
    - selected_features (set): Set of selected features.
    """
    # 0. Replacing inf values
    train_data.replace([np.inf, -np.inf], np.nan, inplace=True)

    # 1. Variance Threshold
    if variance_threshold > 0:
        numerical_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
        numerical_cols = [col for col in numerical_cols if col != id_column]
        selector = VarianceThreshold(threshold=variance_threshold)
        selector.fit(train_data[numerical_cols])
        cols_to_keep_vt = set(train_data[numerical_cols].columns[selector.get_support(indices=True)].tolist())
        print(f"Variables after Variance Threshold: {len(cols_to_keep_vt)}")
    else:
        cols_to_keep_vt = set(train_data.columns)

    # Filtra las columnas basadas en la varianza
    train_data_filtered_vt = train_data[list(cols_to_keep_vt)]

    # 2. Missing Values
    if max_missing_percentage > 0:
        # Calcula el porcentaje de valores faltantes para cada columna
        missing_percentage = train_data_filtered_vt.isnull().mean()
        # Identifica las columnas que tienen un porcentaje de valores faltantes mayor que el máximo permitido
        cols_to_drop_na = missing_percentage[missing_percentage > max_missing_percentage].index
        # Mantén las columnas que tienen un porcentaje de valores faltantes menor o igual al máximo permitido
        cols_to_keep_na = set(train_data_filtered_vt.columns) - set(cols_to_drop_na)
        print(f"Variables after Missing Values Threshold: {len(cols_to_keep_na)}")
    else:
        cols_to_keep_na = set(train_data_filtered_vt.columns)

    # Filtra las columnas basadas en valores faltantes
    train_data_filtered_na = train_data_filtered_vt[list(cols_to_keep_na)]

    # 3. Correlation Matrix Elimination
    if correlation_threshold > 0:
        correlated_features = set()
        correlation_matrix = train_data_filtered_na.corr().abs()
        upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
        for column in upper_triangle.columns:
            if any(upper_triangle[column] > correlation_threshold):
                correlated_features.add(column)
        cols_to_keep_corr = set(train_data_filtered_na.columns) - set(correlated_features)
        print(f"Variables after Correlation Threshold: {len(cols_to_keep_corr)}")
    else:
        cols_to_keep_corr = set(train_data_filtered_na.columns)

    # Devuelve la intersección de las características seleccionadas por varianza, valores faltantes y correlación
    selected_features = cols_to_keep_corr.union(set([id_column]))
    print(f"Total selected features: {len(selected_features)}")

    return selected_features
​
def downcast_df_int_columns(dfname, df):
    list_of_columns = list(df.select_dtypes(include=["int32", "int64"]).columns)
​
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting integers for:", list_of_columns, "\n")
​
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="integer")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
​
    gc.collect()
​
    print(dfname, " - 1. Done")
​
def downcast_df_float_columns(dfname, df):
    list_of_columns = list(df.select_dtypes(include=["float64"]).columns)
​
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting float for:", list_of_columns, "\n")
​
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="float")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
​
    gc.collect()
​
    print(dfname, " - 2. Done")
​
def convert_columns_to_catg(dfname, df, column_list):
    for col in column_list:
        print("converting", col.ljust(30), "size: ", round(df[col].memory_usage(deep=True)*1e-6,2), end="\t")
        df[col] = df[col].astype("category")
        print("->\t", round(df[col].memory_usage(deep=True)*1e-6,2))
    print(dfname, " - 3. Done")
​
def compress_dset (dfname, df):
    downcast_df_int_columns(dfname, df)
    downcast_df_float_columns(dfname, df)


## Get size of objects
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)
​
# List objects and their sizes
def list_objects_size():
    global_vars = globals().items()
    var_sizes = {var: sys.getsizeof(value) for var, value in global_vars}
    sorted_vars = sorted(var_sizes.items(), key=lambda x: x[1], reverse=True)
    for var, size in sorted_vars[0:100]:
        print(f"{var}: {size} bytes")

# Preprocesamiento

## Paso 1: Agrupación de datos de OpportunityPostVenta por OpportunityID

In [ ]:
# Lectura de datos
OpportunityItemPostVenta_prepro = pd.read_csv('gs://ferreyros-datathon-bucket-01/OpportunityItemPostVenta.csv')

In [ ]:
# Conversión
OpportunityItemPostVenta_prepro['OpportunityID'] = OpportunityItemPostVenta_prepro['OpportunityID'].astype('object')

# Registros Duplicados

print(f"Registros antes de eliminar duplicados: {OpportunityItemPostVenta_prepro.shape[0]}")
OpportunityItemPostVenta_prepro.drop_duplicates(inplace=True)
print(f"Registros después de eliminar duplicados: {OpportunityItemPostVenta_prepro.shape[0]}")

In [ ]:
fx_porc_missings(OpportunityItemPostVenta_prepro, only_missings=True)

In [ ]:
# Creación de variables flags (_missing)
create_columns_based_nulls(OpportunityItemPostVenta_prepro,exclude_cols=None,name='field')

# Remover valores nulos
OpportunityItemPostVenta_prepro['ProductModel'].fillna('DESCONOCIDO', inplace=True)
OpportunityItemPostVenta_prepro['ProductBrand'].fillna('DESCONOCIDO', inplace=True)
OpportunityItemPostVenta_prepro['ProductProductCategory'].fillna('DESCONOCIDO', inplace=True)
OpportunityItemPostVenta_prepro['ProductLine'].fillna('DESCONOCIDO', inplace=True)
OpportunityItemPostVenta_prepro['ProductMarket'].fillna('DESCONOCIDO', inplace=True)

OpportunityItemPostVenta_prepro['QuantityProduct'].fillna(OpportunityItemPostVenta_prepro['QuantityProduct'].median(), inplace=True)
OpportunityItemPostVenta_prepro['ValueNegotiatedUSD'].fillna(OpportunityItemPostVenta_prepro['ValueNegotiatedUSD'].median(), inplace=True)

Eliminación de variables:

- **ValueNegotiatedCurrency**. No agregará información para el modelo ya que se trabaja únicamente con el tipo de moneda: 'USD'.
- **ValueNegotiated**. Se decide trabajar con montos en dólares.
- **DocumentClassificationDescription**. Se debe eliminar ya que es una variable de varianza cero. Todos sus valores son iguales. Columna constante.

In [ ]:
OpportunityItemPostVenta_prepro.drop(columns=['ValueNegotiatedCurrency', 'ValueNegotiated'], inplace=True)
OpportunityItemPostVenta_prepro.drop(columns=['DocumentClassificationDescription'], inplace=True)

fx_porc_missings(OpportunityItemPostVenta_prepro, only_missings=True)



**ProductID** no se imputa ya que son identificadores únicos.

Vamos a agrupar las categorías de cada variable según su frecuencia de participación. Hubiese sido valioso hacerlo por el Target sin embargo no contamos con esa información en esta tabla

In [ ]:
# Agrupar las variables

# Definir los conjuntos de códigos

codigo_agd_dict = {
    'ProductModel': ['DESCONOCIDO','Modelo3','Modelo17','Modelo85','Modelo53','Modelo83','Modelo101','Modelo15','Modelo55','Modelo54'],
    'ProductProductCategory': ['DESCONOCIDO','Categoria22','Categoria18','Categoria7','Categoria9','Categoria8','Categoria21','Categoria14','Categoria4','Categoria5'],
    'ProductLine': ['DESCONOCIDO','Linea2','Linea3','Linea1','Linea19','Linea5','Linea18'],
    'ProductMarket': ['DESCONOCIDO','Mercado7','Mercado2','Mercado10','Mercado9','Mercado8']
}

# Función para aplicar las reglas
def codigo_agd_funcion(x, col_name):
    if x in codigo_agd_dict[col_name]:
        return x
    else:
        if col_name == 'ProductModel':
            return 'OTROS_MODELOS'
        elif col_name == 'ProductProductCategory':
            return 'OTRAS_CATEGORIAS'
        elif col_name == 'ProductLine':
            return 'OTRAS_LINEAS'
        elif col_name == 'ProductMarket':
            return 'OTROS_MERCADOS'

# Aplicar las reglas a cada columna
OpportunityItemPostVenta_prepro['ProductModel'] = OpportunityItemPostVenta_prepro['ProductModel'].apply(lambda x: codigo_agd_funcion(x, 'ProductModel'))
OpportunityItemPostVenta_prepro['ProductProductCategory'] = OpportunityItemPostVenta_prepro['ProductProductCategory'].apply(lambda x: codigo_agd_funcion(x, 'ProductProductCategory'))
OpportunityItemPostVenta_prepro['ProductLine'] = OpportunityItemPostVenta_prepro['ProductLine'].apply(lambda x: codigo_agd_funcion(x, 'ProductLine'))
OpportunityItemPostVenta_prepro['ProductMarket'] = OpportunityItemPostVenta_prepro['ProductMarket'].apply(lambda x: codigo_agd_funcion(x, 'ProductMarket'))

# Reemplazo para ProductBrand
OpportunityItemPostVenta_prepro['ProductBrand'] = OpportunityItemPostVenta_prepro['ProductBrand'].replace(['Marca2', 'Marca3'], 'Marca1')

Creación de la variable **ValuePerUnit** y de otras tomando en cuenta esta nueva variable creada

In [ ]:
# Feature Engineering

OpportunityItemPostVenta_prepro['ValueNegotiatedUSD_is_zero'] = (OpportunityItemPostVenta_prepro['ValueNegotiatedUSD'] == 0).astype(int)

min_value_neg_usd = OpportunityItemPostVenta_prepro[OpportunityItemPostVenta_prepro['ValueNegotiatedUSD'] > 0]['ValueNegotiatedUSD'].min()
OpportunityItemPostVenta_prepro['ValueNegotiatedUSD'].replace(0, min_value_neg_usd, inplace=True)

min_value_qp = OpportunityItemPostVenta_prepro[OpportunityItemPostVenta_prepro['QuantityProduct'] > 0]['QuantityProduct'].min()
OpportunityItemPostVenta_prepro['QuantityProduct'].replace(0, min_value_qp, inplace=True)

OpportunityItemPostVenta_prepro['ValuePerUnit']=OpportunityItemPostVenta_prepro['ValueNegotiatedUSD']/OpportunityItemPostVenta_prepro['QuantityProduct']
#OpportunityItemPostVenta_prepro['ValuePerUnit'].fillna(0, inplace=True)

OpportunityItemPostVenta_prepro['PropQuantityProduct'] = OpportunityItemPostVenta_prepro['QuantityProduct'] / OpportunityItemPostVenta_prepro.groupby('OpportunityID')['QuantityProduct'].transform('sum')
OpportunityItemPostVenta_prepro['PropValueNegotiatedUSD'] = OpportunityItemPostVenta_prepro['ValueNegotiatedUSD'] / OpportunityItemPostVenta_prepro.groupby('OpportunityID')['ValueNegotiatedUSD'].transform('sum')

OpportunityItemPostVenta_prepro['LogQuantityProduct'] = np.log1p(OpportunityItemPostVenta_prepro['QuantityProduct'])
OpportunityItemPostVenta_prepro['LogValueNegotiatedUSD'] = np.log1p(OpportunityItemPostVenta_prepro['ValueNegotiatedUSD'])
OpportunityItemPostVenta_prepro['LogValuePerUnit'] = np.log1p(OpportunityItemPostVenta_prepro['ValuePerUnit'])

OpportunityItemPostVenta_prepro['CumSumQuantityProduct'] = OpportunityItemPostVenta_prepro.groupby('OpportunityID')['QuantityProduct'].cumsum()
OpportunityItemPostVenta_prepro['CumSumValueNegotiatedUSD'] = OpportunityItemPostVenta_prepro.groupby('OpportunityID')['ValueNegotiatedUSD'].cumsum()
#OpportunityItemPostVenta_prepro['CumMeanValueNegotiatedUSD'] = OpportunityItemPostVenta_prepro.groupby('OpportunityID')['ValueNegotiatedUSD'].cummean()
#OpportunityItemPostVenta_prepro['CumMeanQuantityProduct'] = OpportunityItemPostVenta_prepro.groupby('OpportunityID')['QuantityProduct'].cummean()

OpportunityItemPostVenta_prepro['RelValuePerUnit_ProductModel'] = OpportunityItemPostVenta_prepro['ValuePerUnit'] / OpportunityItemPostVenta_prepro.groupby('ProductModel')['ValuePerUnit'].transform('mean')
OpportunityItemPostVenta_prepro['RelValuePerUnit_ProductBrand'] = OpportunityItemPostVenta_prepro['ValuePerUnit'] / OpportunityItemPostVenta_prepro.groupby('ProductBrand')['ValuePerUnit'].transform('mean')
OpportunityItemPostVenta_prepro['RelValuePerUnit_ProductProductCategory'] = OpportunityItemPostVenta_prepro['ValuePerUnit'] / OpportunityItemPostVenta_prepro.groupby('ProductProductCategory')['ValuePerUnit'].transform('mean')
OpportunityItemPostVenta_prepro['RelValuePerUnit_ProductLine'] = OpportunityItemPostVenta_prepro['ValuePerUnit'] / OpportunityItemPostVenta_prepro.groupby('ProductLine')['ValuePerUnit'].transform('mean')
OpportunityItemPostVenta_prepro['RelValuePerUnit_ProductMarket'] = OpportunityItemPostVenta_prepro['ValuePerUnit'] / OpportunityItemPostVenta_prepro.groupby('ProductMarket')['ValuePerUnit'].transform('mean')

In [ ]:
fx_porc_missings(OpportunityItemPostVenta_prepro, only_missings=True)

Se generar agrupaciones tomando en cuenta las variables cualitativas y variables cuantitativas. Se generan agregaciones entre cada una de las categorías de las variables cualitativas y las variables cuantitativas

In [ ]:
# Feature Engineering

quantitative_vars = ['QuantityProduct', 'ValueNegotiatedUSD']

flag_vars = ['ProductID_missing_field', 'ProductModel_missing_field', 'ProductBrand_missing_field', 'ProductProductCategory_missing_field',
             'ProductLine_missing_field', 'ProductMarket_missing_field', 'QuantityProduct_missing_field', 'ValueNegotiated_missing_field',
             'ValueNegotiatedCurrency_missing_field', 'ValueNegotiatedUSD_missing_field']

new_quantitative_vars = ['ValuePerUnit', 'PropQuantityProduct', 'PropValueNegotiatedUSD', 'LogQuantityProduct',
                         'LogValueNegotiatedUSD', 'LogValuePerUnit', 'CumSumQuantityProduct', 'CumSumValueNegotiatedUSD',
                         'RelValuePerUnit_ProductModel', 'RelValuePerUnit_ProductBrand', 'RelValuePerUnit_ProductProductCategory',
                         'RelValuePerUnit_ProductLine', 'RelValuePerUnit_ProductMarket','ValueNegotiatedUSD_is_zero']

quantitative_vars = quantitative_vars + new_quantitative_vars #+ flag_vars

# Lista de variables cualitativas
qualitative_vars = ['ProductModel', 'ProductBrand', 'ProductProductCategory', 'ProductLine', 'ProductMarket']

# Diccionario para almacenar los resultados de las agregaciones
agg_results = {}

# Agregación general de 'count'
general_count_agg = OpportunityItemPostVenta_prepro.groupby('OpportunityID').size().reset_index()
general_count_agg.columns = ['OpportunityID', 'count_general_opportunity_id']

# Agregación de 'count' para las variables cualitativas
for qual_var in qualitative_vars:
    qual_vars_count_agg = OpportunityItemPostVenta_prepro.groupby(['OpportunityID', qual_var])[qual_var].agg(['count']).unstack().reset_index()
    qual_vars_count_agg.columns = ['OpportunityID'] + [f'{col[0]}_{col[1]}_{qual_var}' for col in qual_vars_count_agg.columns[1:]]
    agg_results[f'{qual_var}_count'] = qual_vars_count_agg

# Realizar las agregaciones para cada variable cuantitativa
for quant_var in quantitative_vars:
    # Agrupaciones generales
    general_others_agg = OpportunityItemPostVenta_prepro.groupby('OpportunityID')[quant_var].agg(['sum', 'mean', 'median', 'std', 'min', 'max']).reset_index()
    prefixes = ['sum', 'mean', 'median', 'std', 'min', 'max']
    general_others_agg.columns = ['OpportunityID'] + [f'{prefix}_general_{quant_var}' for prefix in prefixes]
    agg_results[f'general_{quant_var}'] = general_others_agg

    # Realizar las agregaciones para cada variable cualitativa
    for qual_var in qualitative_vars:
        agg = OpportunityItemPostVenta_prepro.groupby(['OpportunityID', qual_var])[quant_var].agg(['sum', 'mean', 'median', 'std', 'min', 'max']).unstack().reset_index()
        agg.columns = ['OpportunityID'] + [f'{col[0]}_{col[1]}_{qual_var}_{quant_var}' for col in agg.columns[1:]]
        agg_results[f'{qual_var}_{quant_var}'] = agg

# Combinar todas las tablas en un solo DataFrame
OpportunityItemPostVenta_prepro_agg = pd.DataFrame({'OpportunityID': OpportunityItemPostVenta_prepro['OpportunityID'].unique()})

# Añadir la agregación de 'count' general al DataFrame combinado
OpportunityItemPostVenta_prepro_agg = OpportunityItemPostVenta_prepro_agg.merge(general_count_agg, on='OpportunityID', how='left')

# Añadir las otras agregaciones al DataFrame combinado
for key, df in agg_results.items():
    OpportunityItemPostVenta_prepro_agg = OpportunityItemPostVenta_prepro_agg.merge(df, on='OpportunityID', how='left')

# Manejar NaNs en las tablas agregadas
#OpportunityItemPostVenta_prepro_agg = OpportunityItemPostVenta_prepro_agg.fillna(9999)

In [ ]:
print(OpportunityItemPostVenta_prepro_agg.shape)
print(OpportunityItemPostVenta_prepro_agg.dtypes.unique())
print(OpportunityItemPostVenta_prepro_agg.dtypes[OpportunityItemPostVenta_prepro_agg.dtypes == 'O'])

In [ ]:
#compress_dset("OpportunityItemPostVenta_prepro_agg", OpportunityItemPostVenta_prepro_agg)

In [ ]:
#list_objects_size()
del agg,df,qual_vars_count_agg,general_others_agg,general_count_agg,agg_results,OpportunityItemPostVenta_prepro

## Paso 2: Obtener Insights de X_train_df y X_test_df

In [ ]:
# Cargar los datos
X_train_df_prepro = pd.read_csv('gs://ferreyros-datathon-bucket-01/X_train_df.csv')
X_test_df_prepro = pd.read_csv('gs://ferreyros-datathon-bucket-01/X_test_df.csv')

In [ ]:
print(X_train_df_prepro.shape)
print(X_test_df_prepro.shape)

In [ ]:
# Conversión de datos
X_train_df_prepro['OpportunityID'] = X_train_df_prepro['OpportunityID'].astype('object')
X_train_df_prepro['OpportunityStartDate'] = pd.to_datetime(X_train_df_prepro['OpportunityStartDate'])

X_test_df_prepro['OpportunityID'] = X_test_df_prepro['OpportunityID'].astype('object')
X_test_df_prepro['OpportunityStartDate'] = pd.to_datetime(X_test_df_prepro['OpportunityStartDate'])

In [ ]:
# Remover valores nulos
variables_moda = ['CustomerClassDescription', 'CustomerTypeDescription','CustomerMainMarketDescription',
                  'CustomerEconomicActivityDescription','CustomerAddressDepartmentDescription']

for col in variables_moda:
    moda = X_train_df_prepro[col].mode()[0]
    X_train_df_prepro[col].fillna(moda, inplace=True)


create_columns_based_nulls(X_train_df_prepro,exclude_cols=['CustomerID'],name='field')
X_train_df_prepro['CustomerTaxpayerTypeDescription'].fillna('DESCONOCIDO', inplace=True)


for col in variables_moda:
    moda = X_train_df_prepro[col].mode()[0]
    X_test_df_prepro[col].fillna(moda, inplace=True)

create_columns_based_nulls(X_test_df_prepro,exclude_cols=['CustomerID'],name='field')
X_test_df_prepro['CustomerTaxpayerTypeDescription'].fillna('DESCONOCIDO', inplace=True)

In [ ]:
# Agrupar las categorías tomando en cuenta la conversión con el target

# Diccionario de configuraciones
config = {
    'LeadSourceDescription': {
        'values': ['Fuente10', 'Fuente1', 'Fuente9', 'Fuente8'],
        'label': 'OTRAS_FUENTES'
    },
    'OpportunitySalesRepGroupDescription': {
        'replace': {'Cuenta1': 'Cuenta5'}
    },
    'OpportunitySalesOfficeDescription': {
        'values': ['Oficina5', 'Oficina11', 'Oficina9', 'Oficina2', 'Oficina14', 'Oficina12', 'Oficina3', 'Oficina1', 'Oficina8', 'Oficina7', 'Oficina17', 'Oficina4', 'Oficina6'],
        'label': 'OTRAS_OFICINAS'
    },
    'CustomerTypeDescription': {
        'replace': {'Tipo10': 'Tipo17', 'Tipo11': 'Tipo2'},
        'values': ['Tipo2', 'Tipo16', 'Tipo4', 'Tipo15', 'Tipo14', 'Tipo17', 'Tipo3', 'Tipo9'],
        'label': 'OTROS_TIPOS'
    },
    'CustomerMainMarketDescription': {
        'replace': {'MercadoCliente7': 'MercadoCliente5'},
        'values': ['MercadoCliente9', 'MercadoCliente2', 'MercadoCliente10', 'MercadoCliente14', 'MercadoCliente4', 'MercadoCliente12', 'MercadoCliente11', 'MercadoCliente5', 'MercadoCliente3', 'MercadoCliente15'],
        'label': 'OTROS_MERCADOS'
    },
    'CustomerVerticalIndustryDescription': {
        'replace': {'Industria4': 'Industria1'}
    },
    'CustomerTaxpayerTypeDescription': {
        'replace': {'TipoEmpresa21': 'TipoEmpresa17', 'TipoEmpresa22': 'TipoEmpresa17', 'TipoEmpresa9': 'TipoEmpresa1', 'TipoEmpresa23': 'TipoEmpresa1'},
        'values': ['TipoEmpresa17', 'TipoEmpresa2', 'TipoEmpresa1', 'TipoEmpresa19', 'DESCONOCIDO', 'TipoEmpresa14', 'TipoEmpresa16', 'TipoEmpresa7', 'TipoEmpresa18'],
        'label': 'OTROS_TIPOS_EMPRESAS'
    },
    'CustomerAddressDepartmentDescription': {
        'replace': {'Departamento10': 'Departamento1', 'Departamento22': 'Departamento14'},
        'values': ['Departamento1', 'Departamento14', 'Departamento6', 'Departamento5', 'Departamento7', 'Departamento11', 'Departamento20', 'Departamento15', 'Departamento2', 'Departamento4', 'Departamento3', 'Departamento12', 'Departamento18', 'Departamento17', 'Departamento13', 'Departamento8'],
        'label': 'OTROS_DEPARTAMENTOS'
    },
    'CustomerEconomicActivityDescription': {
        'replacements': {
            'top_1_Activity': ['Actividad95', 'Actividad13', 'Actividad6', 'Actividad100', 'Actividad82', 'Actividad93', 'Actividad106'],
            'top_2_Activity': ['Actividad12', 'Actividad7', 'Actividad46', 'Actividad8', 'Actividad52', 'Actividad53', 'Actividad49', 'Actividad54', 'Actividad4', 'Actividad101', 'Actividad44'],
            'top_3_Activity': ['Actividad17', 'Actividad91', 'Actividad40', 'Actividad51', 'Actividad104', 'Actividad68', 'Actividad118', 'Actividad105', 'Actividad94', 'Actividad42', 'Actividad16', 'Actividad89', 'Actividad90', 'Actividad102', 'Actividad63', 'Actividad80', 'Actividad38', 'Actividad98', 'Actividad73', 'Actividad87', 'Actividad31'],
            'top_4_Activity': ['Actividad92', 'Actividad86', 'Actividad50', 'Actividad71', 'Actividad74', 'Actividad79', 'Actividad36', 'Actividad45', 'Actividad103', 'Actividad23', 'Actividad83', 'Actividad109', 'Actividad1', 'Actividad56', 'Actividad24', 'Actividad66', 'Actividad64', 'Actividad110', 'Actividad34', 'Actividad30', 'Actividad81', 'Actividad37', 'Actividad28', 'Actividad116'],
            'top_5_Activity': ['Actividad59', 'Actividad35', 'Actividad29', 'Actividad120', 'Actividad70', 'Actividad113', 'Actividad27', 'Actividad2', 'Actividad25', 'Actividad72', 'Actividad22', 'Actividad39', 'Actividad84', 'Actividad11', 'Actividad9', 'Actividad76', 'Actividad55'],
            'top_6_Activity': ['Actividad85', 'Actividad69', 'Actividad88', 'Actividad96', 'Actividad62', 'Actividad115', 'Actividad112', 'Actividad60', 'Actividad15'],
            'top_7_Activity': ['Actividad48', 'Actividad47', 'Actividad61', 'Actividad14', 'Actividad121', 'Actividad41', 'Actividad97', 'Actividad77', 'Actividad43', 'Actividad75', 'Actividad65', 'Actividad99']
        },
        'values': ['top_1_Activity', 'top_2_Activity', 'top_3_Activity', 'top_4_Activity', 'top_5_Activity', 'top_6_Activity', 'top_7_Activity'],
        'label': 'top_8_Activity'
    }
}

# Función para aplicar las reglas
def apply_rules(df, config):
    for col, rules in config.items():
        if 'replace' in rules:
            df[col] = df[col].replace(rules['replace'])
        if 'replacements' in rules:
            for new_value, old_values in rules['replacements'].items():
                df[col] = df[col].replace(old_values, new_value)
        if 'values' in rules:
            values = rules['values']
            label = rules['label']
            df[col] = df[col].apply(lambda x: x if x in values else label)
    return df

# Aplicar las reglas a los DataFrames
X_train_df_prepro = apply_rules(X_train_df_prepro, config)
X_test_df_prepro = apply_rules(X_test_df_prepro, config)

In [ ]:
# Feature Engineering
from pandas.tseries.offsets import DateOffset

# Función para crear características basadas en la fecha de oportunidad
def create_date_features(df):
    df['Year'] = df['OpportunityStartDate'].dt.year
    df['Quarter'] = df['OpportunityStartDate'].dt.quarter
    df['DayOfWeek'] = df['OpportunityStartDate'].dt.dayofweek
    df['DayOfYear'] = df['OpportunityStartDate'].dt.dayofyear
    df['Period'] = df['OpportunityStartDate'].dt.strftime('%Y%m')
    df['Month'] = df['OpportunityStartDate'].dt.month
    df['WeekOfYear'] = df['OpportunityStartDate'].dt.isocalendar().week
    df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
    df['IsMonthEnd'] = df['OpportunityStartDate'].dt.is_month_end.astype(int)
    df['IsQuarterEnd'] = df['OpportunityStartDate'].dt.is_quarter_end.astype(int)
    df['IsYearEnd'] = df['OpportunityStartDate'].dt.is_year_end.astype(int)

    today = pd.Timestamp.today()
    df['DaysSinceToday'] = (today - df['OpportunityStartDate']).dt.days

    start_of_year = pd.to_datetime(df['OpportunityStartDate'].dt.year.astype(str) + '-01-01')
    df['DaysSinceStartOfYear'] = (df['OpportunityStartDate'] - start_of_year).dt.days

    df['DaysSinceStartOfQuarter'] = (df['OpportunityStartDate'] - df['OpportunityStartDate'].dt.to_period("Q").dt.start_time).dt.days

    start_of_month = pd.to_datetime(df['OpportunityStartDate'].dt.year.astype(str) + '-' + df['OpportunityStartDate'].dt.month.astype(str) + '-01')
    df['DaysSinceStartOfMonth'] = (df['OpportunityStartDate'] - start_of_month).dt.days

    return df

# Función para crear características basadas en los IDs de clientes y equipos
def create_id_features(df):
    customer_opportunity_count = df.groupby('CustomerID').size().reset_index(name='CustomerOpportunityCount')
    df = df.merge(customer_opportunity_count, on='CustomerID', how='left')

    equipment_opportunity_count = df.groupby('EquipmentSerialNumber').size().reset_index(name='EquipmentOpportunityCount')
    df = df.merge(equipment_opportunity_count, on='EquipmentSerialNumber', how='left')

    return df

# Función para crear características de agregaciones temporales
def create_temporal_aggregations(df):
    last_month = df['OpportunityStartDate'].max() - DateOffset(months=1)
    last_quarter = df['OpportunityStartDate'].max() - DateOffset(months=3)
    last_year = df['OpportunityStartDate'].max() - DateOffset(years=1)

    df['OpportunitiesLastMonth'] = df['OpportunityStartDate'].apply(lambda x: sum((df['OpportunityStartDate'] >= last_month) & (df['OpportunityStartDate'] <= x)))
    df['OpportunitiesLastQuarter'] = df['OpportunityStartDate'].apply(lambda x: sum((df['OpportunityStartDate'] >= last_quarter) & (df['OpportunityStartDate'] <= x)))
    df['OpportunitiesLastYear'] = df['OpportunityStartDate'].apply(lambda x: sum((df['OpportunityStartDate'] >= last_year) & (df['OpportunityStartDate'] <= x)))

    return df

# Función para crear interacciones entre variables
def create_interaction_features(df):
    df['DocClass_CustClass'] = df['OpportunityDocumentClassificationDescription'] + '_' + df['CustomerClassDescription']
    df['DocClass_LeadSource'] = df['OpportunityDocumentClassificationDescription'] + '_' + df['LeadSourceDescription']
    df['DocClass_SalesRepGroup'] = df['OpportunityDocumentClassificationDescription'] + '_' + df['OpportunitySalesRepGroupDescription']
    #df['DocClass_SalesOffice'] = df['OpportunityDocumentClassificationDescription'] + '_' + df['OpportunitySalesOfficeDescription']

    df['CustClass_Type'] = df['CustomerClassDescription'] + '_' + df['CustomerTypeDescription']
    df['CustClass_Market'] = df['CustomerClassDescription'] + '_' + df['CustomerMainMarketDescription']
    df['CustClass_EconActivity'] = df['CustomerClassDescription'] + '_' + df['CustomerEconomicActivityDescription']
    df['CustClass_Industry'] = df['CustomerClassDescription'] + '_' + df['CustomerVerticalIndustryDescription']

    df['LeadSource_SalesRepGroup'] = df['LeadSourceDescription'] + '_' + df['OpportunitySalesRepGroupDescription']
    df['LeadSource_SalesOffice'] = df['LeadSourceDescription'] + '_' + df['OpportunitySalesOfficeDescription'] #ok
    #df['LeadSource_Type'] = df['LeadSourceDescription'] + '_' + df['CustomerTypeDescription']
    #df['LeadSource_Market'] = df['LeadSourceDescription'] + '_' + df['CustomerMainMarketDescription']

    df['SalesRepGroup_SalesOffice'] = df['OpportunitySalesRepGroupDescription'] + '_' + df['OpportunitySalesOfficeDescription'] #ok
    df['SalesRepGroup_Type'] = df['OpportunitySalesRepGroupDescription'] + '_' + df['CustomerTypeDescription'] #ok
    df['SalesRepGroup_Market'] = df['OpportunitySalesRepGroupDescription'] + '_' + df['CustomerMainMarketDescription'] #ok
    df['SalesRepGroup_EconActivity'] = df['OpportunitySalesRepGroupDescription'] + '_' + df['CustomerEconomicActivityDescription']

    #df['SalesOffice_Type'] = df['OpportunitySalesOfficeDescription'] + '_' + df['CustomerTypeDescription']
    #df['SalesOffice_Market'] = df['OpportunitySalesOfficeDescription'] + '_' + df['CustomerMainMarketDescription']
    #df['SalesOffice_EconActivity'] = df['OpportunitySalesOfficeDescription'] + '_' + df['CustomerEconomicActivityDescription']

    #df['Type_Market'] = df['CustomerTypeDescription'] + '_' + df['CustomerMainMarketDescription']
    #df['Type_EconActivity'] = df['CustomerTypeDescription'] + '_' + df['CustomerEconomicActivityDescription']
    df['Type_Industry'] = df['CustomerTypeDescription'] + '_' + df['CustomerVerticalIndustryDescription']
    #df['Type_Taxpayer'] = df['CustomerTypeDescription'] + '_' + df['CustomerTaxpayerTypeDescription']

    #df['Market_EconActivity'] = df['CustomerMainMarketDescription'] + '_' + df['CustomerEconomicActivityDescription']
    df['Market_Industry'] = df['CustomerMainMarketDescription'] + '_' + df['CustomerVerticalIndustryDescription']
    #df['Market_Taxpayer'] = df['CustomerMainMarketDescription'] + '_' + df['CustomerTaxpayerTypeDescription']
    #df['Market_Department'] = df['CustomerMainMarketDescription'] + '_' + df['CustomerAddressDepartmentDescription']

    df['EconActivity_Industry'] = df['CustomerEconomicActivityDescription'] + '_' + df['CustomerVerticalIndustryDescription']
    #df['EconActivity_Taxpayer'] = df['CustomerEconomicActivityDescription'] + '_' + df['CustomerTaxpayerTypeDescription']
    #df['EconActivity_Department'] = df['CustomerEconomicActivityDescription'] + '_' + df['CustomerAddressDepartmentDescription']

    df['Industry_Taxpayer'] = df['CustomerVerticalIndustryDescription'] + '_' + df['CustomerTaxpayerTypeDescription']
    #df['Industry_Department'] = df['CustomerVerticalIndustryDescription'] + '_' + df['CustomerAddressDepartmentDescription']

    #df['Taxpayer_Department'] = df['CustomerTaxpayerTypeDescription'] + '_' + df['CustomerAddressDepartmentDescription']

    return df

# Función para crear características de tendencia de tiempo
def create_trend_features(df):
    first_opportunity = df.groupby('CustomerID')['OpportunityStartDate'].min().reset_index(name='FirstOpportunityDate')
    last_opportunity = df.groupby('CustomerID')['OpportunityStartDate'].max().reset_index(name='LastOpportunityDate')
    df = df.merge(first_opportunity, on='CustomerID', how='left')
    df = df.merge(last_opportunity, on='CustomerID', how='left')

    df['DaysSinceFirstOpportunity'] = (df['OpportunityStartDate'] - df['FirstOpportunityDate']).dt.days
    df['DaysSinceLastOpportunity'] = (df['OpportunityStartDate'] - df['LastOpportunityDate']).dt.days

    df.drop(columns=['FirstOpportunityDate', 'LastOpportunityDate'], inplace=True)

    return df

# Función para aplicar todas las transformaciones
def apply_all_transformations(df):
    df = create_date_features(df)
    df = create_id_features(df)
    df = create_temporal_aggregations(df)
    df = create_interaction_features(df)
    df = create_trend_features(df)
    return df


# Aplicar todas las transformaciones a X_train_df_prepro
X_train_df_prepro = apply_all_transformations(X_train_df_prepro)

# Aplicar todas las transformaciones a X_test_df_prepro
X_test_df_prepro = apply_all_transformations(X_test_df_prepro)

#Print
print(X_train_df_prepro.shape)
print(X_test_df_prepro.shape)

In [ ]:
# Reemplazar los valores nulos finales. El resto será imputado con 9999

median_coc = X_train_df_prepro['CustomerOpportunityCount'].median()
median_dsfo = X_train_df_prepro['DaysSinceFirstOpportunity'].median()
median_dslo = X_train_df_prepro['DaysSinceLastOpportunity'].median()

X_train_df_prepro['CustomerOpportunityCount'].fillna(median_coc, inplace=True)
X_train_df_prepro['DaysSinceFirstOpportunity'].fillna(median_dsfo, inplace=True)
X_train_df_prepro['DaysSinceLastOpportunity'].fillna(median_dslo, inplace=True)

X_test_df_prepro['CustomerOpportunityCount'].fillna(median_coc, inplace=True)
X_test_df_prepro['DaysSinceFirstOpportunity'].fillna(median_dsfo, inplace=True)
X_test_df_prepro['DaysSinceLastOpportunity'].fillna(median_dslo, inplace=True)

In [ ]:
fx_porc_missings(X_train_df_prepro, only_missings=True)
fx_porc_missings(X_test_df_prepro, only_missings=True)

In [ ]:
# Preparar los datos antes de ejecutar un modelo

def preprocess_data(df, is_train=True):
    # Eliminar variables innecesarias
    df = df.drop(columns=['OpportunityStartDate'])

    if is_train:
        # Guardar y luego eliminar la variable objetivo
        target = df['ClosedWonOpportunity']
        df = df.drop(columns=['ClosedWonOpportunity'])

    # Variables booleanas
    df['VentaWeb'] = df['VentaWeb'].astype(int)

    # Variables categóricas
    categorical_vars = [
        'OpportunityDocumentClassificationDescription', 'CustomerClassDescription', 'LeadSourceDescription',
        'OpportunitySalesRepGroupDescription', 'OpportunitySalesOfficeDescription', 'CustomerTypeDescription',
        'CustomerMainMarketDescription', 'CustomerEconomicActivityDescription', 'CustomerVerticalIndustryDescription',
        'CustomerTaxpayerTypeDescription', 'CustomerAddressDepartmentDescription', 'Period',
        'DocClass_CustClass','DocClass_LeadSource','DocClass_SalesRepGroup',
        'CustClass_Type','CustClass_Market','CustClass_EconActivity','CustClass_Industry',
        'LeadSource_SalesRepGroup','LeadSource_SalesOffice',
        'SalesRepGroup_SalesOffice','SalesRepGroup_Type','SalesRepGroup_Market','SalesRepGroup_EconActivity',
        'Type_Industry',
        'Market_Industry',
        'EconActivity_Industry',
        'Industry_Taxpayer']

    # Crear variables dummy asegurándose de que los resultados sean binarios (0 y 1)
    df = pd.get_dummies(df, columns=categorical_vars, drop_first=True, dtype=int)

    if is_train:
        return df, target
    else:
        return df

# Preprocesar los datos de entrenamiento y prueba
X_train_df_prepro, y_train = preprocess_data(X_train_df_prepro, is_train=True)
X_test_df_prepro = preprocess_data(X_test_df_prepro, is_train=False)

# Verificar si las columnas de entrenamiento y prueba son iguales
missing_cols_in_test = set(X_train_df_prepro.columns) - set(X_test_df_prepro.columns)
for col in missing_cols_in_test:
    X_test_df_prepro[col] = 0

missing_cols_in_train = set(X_test_df_prepro.columns) - set(X_train_df_prepro.columns)
for col in missing_cols_in_train:
    X_train_df_prepro[col] = 0

# Asegurar que el orden de las columnas sea el mismo
X_test_df_prepro = X_test_df_prepro[X_train_df_prepro.columns]

# Añadir la variable objetivo al final del conjunto de entrenamiento
X_train_df_prepro['ClosedWonOpportunity'] = y_train

In [ ]:
# Validar los tipos de datos
print(X_train_df_prepro.shape)
print(X_train_df_prepro.dtypes.unique())
print(X_train_df_prepro.dtypes[X_train_df_prepro.dtypes == 'O'])

print("---------------")

# Validar los tipos de datos
print(X_test_df_prepro.shape)
print(X_test_df_prepro.dtypes.unique())
print(X_test_df_prepro.dtypes[X_test_df_prepro.dtypes == 'O'])

In [ ]:
#compress_dset("X_train_df_prepro", X_train_df_prepro)
#compress_dset("X_test_df_prepro", X_test_df_prepro)

In [ ]:
#list_objects_size()
del y_train,missing_cols_in_test,missing_cols_in_train

## Paso 3: Agrupación de datos en EquipmentPostVenta por CurrentCustomerID

In [ ]:
# Lectura de datos
EquipmentPostVenta_prepro = pd.read_csv('gs://ferreyros-datathon-bucket-01/EquipmentPostVenta.csv')

In [ ]:
# Lectura de valores nulos
fx_porc_missings(EquipmentPostVenta_prepro, only_missings=True)

In [ ]:
# Creación de variables flags (_missing)
create_columns_based_nulls(EquipmentPostVenta_prepro,exclude_cols=None,name='field')

# Imputación de valores nulos
EquipmentPostVenta_prepro['BrandDescription'].fillna('DESCONOCIDO', inplace=True)
EquipmentPostVenta_prepro['FamilyDescription'].fillna('DESCONOCIDO', inplace=True)
EquipmentPostVenta_prepro['LineDescription'].fillna('DESCONOCIDO', inplace=True)
EquipmentPostVenta_prepro['MarketDescription'].fillna('DESCONOCIDO', inplace=True)
EquipmentPostVenta_prepro['ApplicationCodeDescription'].fillna('DESCONOCIDO', inplace=True)
EquipmentPostVenta_prepro['Manufacturer'].fillna('DESCONOCIDO', inplace=True)
EquipmentPostVenta_prepro['ItemType'].fillna('DESCONOCIDO', inplace=True)
EquipmentPostVenta_prepro['FamilyDescriptionLLM'].fillna('DESCONOCIDO', inplace=True)

In [ ]:
# Imputación en YearOfProduction
median_year = EquipmentPostVenta_prepro['YearOfProduction'].median()
EquipmentPostVenta_prepro['YearOfProduction'].fillna(median_year, inplace=True)

inconsistent_values = [0.0, 201.0, 202.0]

valid_data = EquipmentPostVenta_prepro[~EquipmentPostVenta_prepro['YearOfProduction'].isin(inconsistent_values)]
median_year_of_production = valid_data['YearOfProduction'].median()
EquipmentPostVenta_prepro['YearOfProduction'] = EquipmentPostVenta_prepro['YearOfProduction'].replace(inconsistent_values, median_year_of_production)

# Creación de la variable Antiguedad
current_year = datetime.now().year
EquipmentPostVenta_prepro['Antiguedad'] = current_year - EquipmentPostVenta_prepro['YearOfProduction']

print("EquipmentPostVenta_prepro")
EquipmentPostVenta_prepro.shape

In [ ]:
# Agrupar las categorías según su frecuencia relativa
# Diccionario de configuraciones
config = {
    'EquipmentTypeDescription': {
        'values': ['EquipoTipo1', 'EquipoTipo8', 'EquipoTipo7'],
        'label': 'OTROS_EQUIPOS_TIPO'
    },
    'BrandDescription': {
        'values': ['Marca9', 'DESCONOCIDO', 'Marca36'],
        'label': 'OTRAS_MARCAS'
    },
    'FamilyDescription': {
        'values': ['Familia18', 'Familia51', 'Familia84', 'Familia87', 'Familia19', 'Familia99', 'Familia86', 'Familia43','Familia89', 'Familia71', 'Familia23', 'Familia31', 'Familia96', 'Familia20', 'Familia54', 'Familia21',
                  'Familia91', 'Familia59', 'Familia48', 'Familia41', 'Familia29', 'Familia56', 'Familia95', 'Familia35','DESCONOCIDO'],
        'label': 'OTRAS_FAMILIAS'
    },
    'LineDescription': {
        'values': ['DESCONOCIDO', 'Linea6', 'Linea3', 'Linea1', 'Linea5', 'Linea8'],
        'label': 'OTRAS_LINEAS'
    },
    'MarketDescription': {
        'values': ['DESCONOCIDO', 'Mercado5', 'Mercado2', 'Mercado9', 'Mercado8', 'Mercado13', 'Mercado12'],
        'label': 'OTROS_MERCADOS'
    },
    'ApplicationCodeDescription': {
        'values': ['DESCONOCIDO', 'TRABAJO MEDIO', 'TRABAJO LIGERO', 'TRABAJO PESADO', 'MOTOR PRINCIPAL', 'GENERADOR STANDBY', 'PRIME'],
        'label': 'OTROS_TIPOS_DE_TRABAJO'
    },
    'Manufacturer': {
        'values': ['Fabricante14', 'Fabricante46'],
        'label': 'OTROS_FABRICANTES'
    },
    'FamilyDescriptionLLM': {
        'values': ['FamiliaDescripcion25', 'FamiliaDescripcion30', 'FamiliaDescripcion4', 'FamiliaDescripcion6', 'FamiliaDescripcion11', 'FamiliaDescripcion7', 'FamiliaDescripcion10',
                  'FamiliaDescripcion9','FamiliaDescripcion15','FamiliaDescripcion24','FamiliaDescripcion22','FamiliaDescripcion14','FamiliaDescripcion26','DESCONOCIDO'],
        'label': 'OTRAS_FAMILIA_DESCRIPCION'
    },
}

# Función para aplicar las reglas
def apply_rules(df, config):
    for col, rules in config.items():
        if 'values' in rules:
            values = rules['values']
            label = rules['label']
            df[col] = df[col].apply(lambda x: x if x in values else label)
    return df

# Aplicar las reglas a los DataFrames
EquipmentPostVenta_prepro = apply_rules(EquipmentPostVenta_prepro, config)

# Reemplazar
EquipmentPostVenta_prepro['ApplicationCodeDescription'] = EquipmentPostVenta_prepro['ApplicationCodeDescription'].replace('TRABAJO MEDIO', 'TRABAJO_MEDIO')
EquipmentPostVenta_prepro['ApplicationCodeDescription'] = EquipmentPostVenta_prepro['ApplicationCodeDescription'].replace('TRABAJO LIGERO', 'TRABAJO_LIGERO')
EquipmentPostVenta_prepro['ApplicationCodeDescription'] = EquipmentPostVenta_prepro['ApplicationCodeDescription'].replace('TRABAJO PESADO', 'TRABAJO_PESADO')
EquipmentPostVenta_prepro['ApplicationCodeDescription'] = EquipmentPostVenta_prepro['ApplicationCodeDescription'].replace('MOTOR PRINCIPAL', 'MOTOR_PRINCIPAL')
EquipmentPostVenta_prepro['ApplicationCodeDescription'] = EquipmentPostVenta_prepro['ApplicationCodeDescription'].replace('GENERADOR STANDBY', 'GENERADOR_STANDBY')

In [ ]:
# Feature Engineering

quantitative_vars = ['YearOfProduction', 'Antiguedad']

flag_vars = ['BrandDescription_missing_field', 'FamilyDescription_missing_field', 'LineDescription_missing_field', 'MarketDescription_missing_field',
             'YearOfProduction_missing_field', 'ApplicationCodeDescription_missing_field', 'Manufacturer_missing_field', 'ItemType_missing_field',
             'FamilyDescriptionLLM_missing_field']

#quantitative_vars = quantitative_vars + flag_vars

# Lista de variables cualitativas
qualitative_vars = ['EquipmentTypeDescription', 'BrandDescription', 'FamilyDescription','LineDescription', 'MarketDescription', 'ApplicationCodeDescription','Manufacturer', 'ItemType', 'FamilyDescriptionLLM']

# Diccionario para almacenar los resultados de las agregaciones
agg_results = {}

# Agregación general de 'count'
general_count_agg = EquipmentPostVenta_prepro.groupby('CurrentCustomerID').size().reset_index()
general_count_agg.columns = ['CurrentCustomerID', 'count_general_customer_id']

# Agregación de 'count' para las variables cualitativas
for qual_var in qualitative_vars:
    qual_vars_count_agg = EquipmentPostVenta_prepro.groupby(['CurrentCustomerID', qual_var])[qual_var].agg(['count']).unstack().reset_index()
    qual_vars_count_agg.columns = ['CurrentCustomerID'] + [f'{col[0]}_{col[1]}_{qual_var}' for col in qual_vars_count_agg.columns[1:]]
    agg_results[f'{qual_var}_count'] = qual_vars_count_agg

# Realizar las agregaciones para cada variable cuantitativa
for quant_var in quantitative_vars:
    # Agrupaciones generales
    general_others_agg = EquipmentPostVenta_prepro.groupby('CurrentCustomerID')[quant_var].agg(['sum', 'mean', 'median', 'std', 'min', 'max']).reset_index()
    prefixes = ['sum', 'mean', 'median', 'std', 'min', 'max']
    general_others_agg.columns = ['CurrentCustomerID'] + [f'{prefix}_general_{quant_var}' for prefix in prefixes]
    agg_results[f'general_{quant_var}'] = general_others_agg

    # Realizar las agregaciones para cada variable cualitativa
    for qual_var in qualitative_vars:
        agg = EquipmentPostVenta_prepro.groupby(['CurrentCustomerID', qual_var])[quant_var].agg(['sum', 'mean', 'median', 'std', 'min', 'max']).unstack().reset_index()
        agg.columns = ['CurrentCustomerID'] + [f'{col[0]}_{col[1]}_{qual_var}_{quant_var}' for col in agg.columns[1:]]
        agg_results[f'{qual_var}_{quant_var}'] = agg

# Combinar todas las tablas en un solo DataFrame
EquipmentPostVenta_prepro_agg = pd.DataFrame({'CurrentCustomerID': EquipmentPostVenta_prepro['CurrentCustomerID'].unique()})

# Añadir la agregación de 'count' general al DataFrame combinado
EquipmentPostVenta_prepro_agg = EquipmentPostVenta_prepro_agg.merge(general_count_agg, on='CurrentCustomerID', how='left')

# Añadir las otras agregaciones al DataFrame combinado
for key, df in agg_results.items():
    EquipmentPostVenta_prepro_agg = EquipmentPostVenta_prepro_agg.merge(df, on='CurrentCustomerID', how='left')

# Manejar NaNs en las tablas agregadas
#OpportunityItemPostVenta_prepro_agg = OpportunityItemPostVenta_prepro_agg.fillna(9999)

In [ ]:
# Validar los tipos de datos
print(EquipmentPostVenta_prepro_agg.shape)
print(EquipmentPostVenta_prepro_agg.dtypes.unique())
print(EquipmentPostVenta_prepro_agg.dtypes[EquipmentPostVenta_prepro_agg.dtypes == 'O'])

In [ ]:
#compress_dset("EquipmentPostVenta_prepro_agg", EquipmentPostVenta_prepro_agg)

In [ ]:
# Convertir a dummy las variables de EquipmentPostVenta_prepro

# Variables categóricas
categorical_vars = ['EquipmentTypeDescription', 'BrandDescription', 'FamilyDescription','LineDescription', 'MarketDescription',
                    'ApplicationCodeDescription','Manufacturer', 'ItemType', 'FamilyDescriptionLLM']

# Crear variables dummy asegurándose de que los resultados sean binarios (0 y 1)
EquipmentPostVenta_prepro = pd.get_dummies(EquipmentPostVenta_prepro, columns=categorical_vars, drop_first=True, dtype=int)

In [ ]:
# Validar los tipos de datos
print(EquipmentPostVenta_prepro.shape)
print(EquipmentPostVenta_prepro.dtypes.unique())
print(EquipmentPostVenta_prepro.dtypes[EquipmentPostVenta_prepro.dtypes == 'O'])

In [ ]:
#compress_dset("EquipmentPostVenta_prepro", EquipmentPostVenta_prepro)

In [ ]:
#list_objects_size()
del valid_data,agg,df,qual_vars_count_agg,general_others_agg,general_count_agg,agg_results

## Paso 4: Agrupación de datos de horometros_vl por SerialNumber y generación de nuevas variables

In [ ]:
# Lectura de datos
horometros_vl_prepro = pd.read_csv('gs://ferreyros-datathon-bucket-01/horometros_vl.csv')

In [ ]:
horometros_vl_prepro.info()

In [ ]:
# Validar la cantidad de repetidos por Month
print(horometros_vl_prepro.shape[0])
print(horometros_vl_prepro.SerialNumber.nunique())

In [ ]:
# Validar
(horometros_vl_prepro["SerialNumber"] + horometros_vl_prepro['Month'].astype(str)).nunique()

Esto demuestra que no se tiene repetidos de **Month** para cada **SerialNumber**

In [ ]:
# Feature Engineering

# Extraer características del Month
horometros_vl_prepro['Month'] = pd.to_datetime(horometros_vl_prepro['Month'])

# Extraer features de Month
horometros_vl_prepro['Year'] = horometros_vl_prepro['Month'].dt.year
horometros_vl_prepro['MonthF'] = horometros_vl_prepro['Month'].dt.month
horometros_vl_prepro['MonthM'] = 'M' + horometros_vl_prepro['MonthF'].astype(str)
horometros_vl_prepro['Quarter'] = horometros_vl_prepro['Month'].dt.quarter
horometros_vl_prepro['QuarterQ'] = 'Q' + horometros_vl_prepro['Quarter'].astype(str)
horometros_vl_prepro['YearQuarter'] = horometros_vl_prepro['Month'].dt.to_period('Q').astype(str)
#horometros_vl_prepro['DayOfWeek'] = horometros_vl_prepro['Month'].dt.dayofweek #0:Monday
horometros_vl_prepro['NameDayOfWeek'] = horometros_vl_prepro['Month'].dt.strftime('%A')
#horometros_vl_prepro['DayOfYear'] = horometros_vl_prepro['Month'].dt.dayofyear
#horometros_vl_prepro['WeekOfYear'] = horometros_vl_prepro['Month'].dt.isocalendar().week
horometros_vl_prepro['Period'] = horometros_vl_prepro['Month'].dt.strftime('%Y%m')
# Día del mes
#horometros_vl_prepro['DayOfMonth'] = horometros_vl_prepro['Month'].dt.day
# Semana del mes
#horometros_vl_prepro['WeekOfMonth'] = horometros_vl_prepro['Month'].apply(lambda x: (x.day-1) // 7 + 1)

# Días desde la primera fecha registrada hasta cada registro (teniendo en cuenta SerialNumber)
horometros_vl_prepro['DaysSinceFirstDate'] = horometros_vl_prepro.groupby('SerialNumber')['Month'].transform(lambda x: (x - x.min()).dt.days)

# Días hasta el final del mes para cada registro
horometros_vl_prepro['DaysUntilEndOfMonth'] = horometros_vl_prepro['Month'].dt.days_in_month - horometros_vl_prepro['Month'].dt.day

# Si la fecha está cerca del inicio o final del mes (primera semana/última semana del mes)
#horometros_vl_prepro['IsBeginningOfMonth'] = (horometros_vl_prepro['Month'].dt.day <= 7).astype(int)
#horometros_vl_prepro['IsEndOfMonth'] = (horometros_vl_prepro['Month'].dt.day >= horometros_vl_prepro['Month'].dt.days_in_month - 7).astype(int)

Contar el número de **SerialNumber** presentes en 1,2,3,4,5,6 años.

In [ ]:
import pandas as pd

unique_years = horometros_vl_prepro['Year'].unique()
print(f'Años únicos: {unique_years}')

pivot_table = horometros_vl_prepro.pivot_table(index='SerialNumber', columns='Year', aggfunc='size', fill_value=0)

serials_counts = (pivot_table > 0).sum(axis=1)
counts_by_years = serials_counts.value_counts().sort_index()

for years, count in counts_by_years.items():
    print(f'Número de SerialNumber presentes en {years} años: {count}')

Cantidad de items por **SerialNumber**

In [ ]:
horometros_vl_prepro.groupby('SerialNumber')['SerialNumber'].count()

In [ ]:
# Max
horometros_vl_prepro.groupby('SerialNumber')['Month'].nunique().max()

In [ ]:
horometros_vl_prepro.loc[horometros_vl_prepro.SerialNumber=='fefd2b07607dbc0f',:]

In [ ]:
# Feature Engineering

quantitative_vars = ['HorasTrabajadas1Mes', 'Horometro','HorasTrabajadas3Meses','HorasTrabajadas6Meses','HorasTrabajadas12Meses','DaysSinceFirstDate','DaysUntilEndOfMonth']

# Lista de variables cualitativas
qualitative_vars = ['Year', 'MonthM', 'QuarterQ', 'YearQuarter','NameDayOfWeek']

# Diccionario para almacenar los resultados de las agregaciones
agg_results = {}

# Agregación general de 'count'
general_count_agg = horometros_vl_prepro.groupby('SerialNumber').size().reset_index()
general_count_agg.columns = ['SerialNumber', 'count_serial_number_id']

# Agregación de 'count' para las variables cualitativas
for qual_var in qualitative_vars:
    qual_vars_count_agg = horometros_vl_prepro.groupby(['SerialNumber', qual_var])[qual_var].agg(['count']).unstack().reset_index()
    qual_vars_count_agg.columns = ['SerialNumber'] + [f'{col[0]}_{col[1]}_{qual_var}' for col in qual_vars_count_agg.columns[1:]]
    agg_results[f'{qual_var}_count'] = qual_vars_count_agg

# Realizar las agregaciones para cada variable cuantitativa
for quant_var in quantitative_vars:
    # Agrupaciones generales
    general_others_agg = horometros_vl_prepro.groupby('SerialNumber')[quant_var].agg(['sum', 'mean', 'median', 'std', 'min', 'max']).reset_index()
    prefixes = ['sum', 'mean', 'median', 'std', 'min', 'max']
    general_others_agg.columns = ['SerialNumber'] + [f'{prefix}_general_{quant_var}' for prefix in prefixes]
    agg_results[f'general_{quant_var}'] = general_others_agg

    # Realizar las agregaciones para cada variable cualitativa
    for qual_var in qualitative_vars:
        agg = horometros_vl_prepro.groupby(['SerialNumber', qual_var])[quant_var].agg(['sum', 'mean', 'median', 'std', 'min', 'max']).unstack().reset_index()
        agg.columns = ['SerialNumber'] + [f'{col[0]}_{col[1]}_{qual_var}_{quant_var}' for col in agg.columns[1:]]
        agg_results[f'{qual_var}_{quant_var}'] = agg

# Combinar todas las tablas en un solo DataFrame
horometros_vl_prepro_agg = pd.DataFrame({'SerialNumber': horometros_vl_prepro['SerialNumber'].unique()})

# Añadir la agregación de 'count' general al DataFrame combinado
horometros_vl_prepro_agg = horometros_vl_prepro_agg.merge(general_count_agg, on='SerialNumber', how='left')

# Añadir las otras agregaciones al DataFrame combinado
for key, df in agg_results.items():
    horometros_vl_prepro_agg = horometros_vl_prepro_agg.merge(df, on='SerialNumber', how='left')

# Agregar más estadísticas si es necesario
# Ejemplo: Coeficiente de variación (std / mean)
horometros_vl_prepro_agg['CoefVarHorasTrabajadas'] = horometros_vl_prepro_agg['std_general_HorasTrabajadas1Mes'] / horometros_vl_prepro_agg['mean_general_HorasTrabajadas1Mes']
horometros_vl_prepro_agg['CoefVarHorometro'] = horometros_vl_prepro_agg['std_general_Horometro'] / horometros_vl_prepro_agg['mean_general_Horometro']

# Manejar NaNs en las tablas agregadas
#OpportunityItemPostVenta_prepro_agg = OpportunityItemPostVenta_prepro_agg.fillna(9999)

#PRINT
horometros_vl_prepro_agg.shape

In [ ]:
# Feature Engineering

# Calcular la diferencia entre la primera y la última fecha y la suma de HorasTrabajadas1Mes
horometros_agg = horometros_vl_prepro.groupby('SerialNumber').agg(
    FirstDate=('Month', 'min'),
    LastDate=('Month', 'max'),
    FirstHorometro=('Horometro', 'first'),
    LastHorometro=('Horometro', 'last'),
    FirstHorasTrabajadas1Mes=('HorasTrabajadas1Mes', 'first'),
    LastHorasTrabajadas1Mes=('HorasTrabajadas1Mes', 'last'),
    FirstHorasTrabajadas3Mes=('HorasTrabajadas3Meses', 'first'),
    LastHorasTrabajadas3Mes=('HorasTrabajadas3Meses', 'last'),
    FirstHorasTrabajadas6Mes=('HorasTrabajadas6Meses', 'first'),
    LastHorasTrabajadas6Mes=('HorasTrabajadas6Meses', 'last'),
    FirstHorasTrabajadas12Mes=('HorasTrabajadas12Meses', 'first'),
    LastHorasTrabajadas12Mes=('HorasTrabajadas12Meses', 'last')
).reset_index()

# Calcular la duración en días y la diferencia en Horometro
horometros_agg['DurationDays'] = (horometros_agg['LastDate'] - horometros_agg['FirstDate']).dt.days
horometros_agg['DiffHorometro'] = horometros_agg['LastHorometro'] - horometros_agg['FirstHorometro']
horometros_agg['DiffLastHorasTrabajadas1Mes'] = horometros_agg['LastHorasTrabajadas1Mes'] - horometros_agg['FirstHorasTrabajadas1Mes']
horometros_agg['DiffLastHorasTrabajadas3Mes'] = horometros_agg['LastHorasTrabajadas3Mes'] - horometros_agg['FirstHorasTrabajadas3Mes']
horometros_agg['DiffLastHorasTrabajadas6Mes'] = horometros_agg['LastHorasTrabajadas6Mes'] - horometros_agg['FirstHorasTrabajadas6Mes']
horometros_agg['DiffLastHorasTrabajadas12Mes'] = horometros_agg['LastHorasTrabajadas12Mes'] - horometros_agg['FirstHorasTrabajadas12Mes']

# Eliminar columnas innecesarias
horometros_agg.drop(columns=['FirstDate', 'LastDate', 'FirstHorometro', 'LastHorometro',
                             'FirstHorasTrabajadas1Mes','LastHorasTrabajadas1Mes',
                             'FirstHorasTrabajadas3Mes','LastHorasTrabajadas3Mes',
                             'LastHorasTrabajadas6Mes','FirstHorasTrabajadas6Mes',
                             'FirstHorasTrabajadas12Mes','LastHorasTrabajadas12Mes'], inplace=True)

horometros_agg.shape

In [ ]:
# Cruzar los Datasets
horometros_vl_prepro_agg = horometros_vl_prepro_agg.merge(horometros_agg,
                                            left_on=['SerialNumber'],
                                            right_on=['SerialNumber'],
                                            how='left')

In [ ]:
# Validar los tipos de datos
print(horometros_vl_prepro_agg.shape)
print(horometros_vl_prepro_agg.dtypes.unique())
print(horometros_vl_prepro_agg.dtypes[horometros_vl_prepro_agg.dtypes == 'O'])

In [ ]:
#compress_dset("EquipmentPostVenta_prepro", EquipmentPostVenta_prepro)

In [ ]:
#list_objects_size()
del horometros_vl_prepro,agg,df,qual_vars_count_agg,pivot_table,general_others_agg,general_count_agg,serials_counts,counts_by_years,agg_results,horometros_agg

## Paso 5: Cruzar X_train_df y X_test_df

In [ ]:
print(OpportunityItemPostVenta_prepro_agg.shape)
print(X_train_df_prepro.shape)
print(X_test_df_prepro.shape)
print(EquipmentPostVenta_prepro.shape)
print(EquipmentPostVenta_prepro_agg.shape)
print(horometros_vl_prepro_agg.shape)

In [ ]:
# Cruzar todos los Datasets

# Realizar el primer cruce en X_train_df_prepro
X_train_df_prepro = X_train_df_prepro.merge(OpportunityItemPostVenta_prepro_agg,
                                            left_on=['OpportunityID'],
                                            right_on=['OpportunityID'],
                                            how='left')

# Realizar el segundo cruce en X_train_df_prepro
X_train_df_prepro = X_train_df_prepro.merge(EquipmentPostVenta_prepro,
                                            left_on=['CustomerID', 'EquipmentSerialNumber'],
                                            right_on=['CurrentCustomerID', 'EquipmentID'],
                                            how='left')

# Realizar el tercer cruce en X_train_df_prepro
X_train_df_prepro = X_train_df_prepro.merge(EquipmentPostVenta_prepro_agg,
                                            left_on=['CustomerID'],
                                            right_on=['CurrentCustomerID'],
                                            how='left')


# Realizar el cuarto cruce en X_train_df_prepro
X_train_df_prepro = X_train_df_prepro.merge(horometros_vl_prepro_agg,
                                            left_on=['EquipmentSerialNumber'],
                                            right_on=['SerialNumber'],
                                            how='left')

# Eliminar las llaves redundantes de X_train_df_prepro
X_train_df_prepro.drop(columns=['CurrentCustomerID_x', 'EquipmentID', 'CurrentCustomerID_y','SerialNumber','EquipmentSerialNumber','CustomerID'], inplace=True)


# Realizar el primer cruce en X_test_df_prepro
X_test_df_prepro = X_test_df_prepro.merge(OpportunityItemPostVenta_prepro_agg,
                                            left_on=['OpportunityID'],
                                            right_on=['OpportunityID'],
                                            how='left')

# Realizar el primer cruce en X_test_df_prepro
X_test_df_prepro = X_test_df_prepro.merge(EquipmentPostVenta_prepro,
                                          left_on=['CustomerID', 'EquipmentSerialNumber'],
                                          right_on=['CurrentCustomerID', 'EquipmentID'],
                                          how='left')

# Realizar el segundo cruce en X_test_df_prepro
X_test_df_prepro = X_test_df_prepro.merge(EquipmentPostVenta_prepro_agg,
                                          left_on=['CustomerID'],
                                          right_on=['CurrentCustomerID'],
                                          how='left')

# Realizar el cuarto cruce en X_test_df_prepro
X_test_df_prepro = X_test_df_prepro.merge(horometros_vl_prepro_agg,
                                            left_on=['EquipmentSerialNumber'],
                                            right_on=['SerialNumber'],
                                            how='left')

# Eliminar las llaves redundantes de X_test_df_prepro
X_test_df_prepro.drop(columns=['CurrentCustomerID_x', 'EquipmentID', 'CurrentCustomerID_y','SerialNumber','EquipmentSerialNumber','CustomerID'], inplace=True)

In [ ]:
# Validar los tipos de datos
print(X_train_df_prepro.shape)
print(X_train_df_prepro.dtypes.unique())
print(X_train_df_prepro.dtypes[X_train_df_prepro.dtypes == 'O'])

print("-------------------------")

print(X_test_df_prepro.shape)
print(X_test_df_prepro.dtypes.unique())
print(X_test_df_prepro.dtypes[X_test_df_prepro.dtypes == 'O'])

In [ ]:
#X_train_df_prepro.drop(['EquipmentSerialNumber','CustomerID'],inplace=True)
#X_test_df_prepro.drop(['EquipmentSerialNumber','CustomerID'],inplace=True)

In [ ]:
#compress_dset("X_train_df_prepro", X_train_df_prepro)
#compress_dset("X_test_df_prepro", X_test_df_prepro)

In [ ]:
#X_train_df_prepro.to_pickle('X_train_df_prepro.pkl')
#X_test_df_prepro.to_pickle('X_test_df_prepro.pkl')

# Selección de variables

In [ ]:
X_train_df_prepro = pd.read_pickle('X_train_df_prepro.pkl')
X_test_df_prepro = pd.read_pickle('X_test_df_prepro.pkl')

In [ ]:
#list_objects_size()
import gc
#del OpportunityItemPostVenta_prepro_agg, horometros_vl_prepro_agg, EquipmentPostVenta_prepro, EquipmentPostVenta_prepro_agg
gc.collect()

In [ ]:
print(X_train_df_prepro.shape)
print(X_test_df_prepro.shape)

In [ ]:
# Validar los tipos de datos
print(X_train_df_prepro.shape)
print(X_train_df_prepro.dtypes.unique())
print(X_train_df_prepro.dtypes[X_train_df_prepro.dtypes == 'O'])

In [ ]:
# Validar los tipos de datos
print(X_test_df_prepro.shape)
print(X_test_df_prepro.dtypes.unique())
print(X_test_df_prepro.dtypes[X_test_df_prepro.dtypes == 'O'])

In [ ]:
# Selección de variables (1era Iteración)
#features_selected_init = feature_selection_customized_initial('OpportunityID', X_train_df_prepro, variance_threshold=0.01, max_missing_percentage=0.5, correlation_threshold=0.8)
#len(features_selected_init)

In [ ]:
# Selección de variables (1era Iteración)
#features_selected_init = feature_selection_customized_last('OpportunityID', X_train_df_prepro, variance_threshold=0.01, max_missing_percentage=0.8, correlation_threshold=0.9)
#len(features_selected_init)

Comprobar si se encuentra el target entre las variables seleccionadas

In [ ]:
print('ClosedWonOpportunity' in features_selected_init)

Selección de variables con RFE y LightGBM

In [ ]:

import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, mutual_info_classif, SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold


# Función de transformación logarítmica
def log_transform(x):
    return np.log1p(x)

# Ajustar conjuntos de datos de entrenamiento
X_train_df_prepro = X_train_df_prepro[list(features_selected_init)]

# Imputar nulos con -99999 aquellos nulos que no cruzaron en el cruce de las tablas
X_train_df_prepro.fillna(-99999, inplace=True)

# Separar variable objetivo
y_train = X_train_df_prepro['ClosedWonOpportunity']
X_train_df_prepro.drop(['ClosedWonOpportunity'], axis=1, inplace=True)
X_train = X_train_df_prepro

# Función para entrenamiento con LightGBM
def train_lgb(X, y, params, num_boost_round=15):
    X_train_df, X_valid_df, y_train_df, y_valid_df = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    numerical_transformer = Pipeline(steps=[
        ('log_transform', FunctionTransformer(log_transform, validate=False)),
        ('scaler', StandardScaler())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, X_train_df.columns)
        ])

    preprocessor.fit(X_train_df)
    X_train_transformed = preprocessor.transform(X_train_df)
    X_valid_transformed = preprocessor.transform(X_valid_df)

    train_set = lgb.Dataset(X_train_transformed, label=y_train_df)
    valid_set = lgb.Dataset(X_valid_transformed, label=y_valid_df, reference=train_set)

    model = lgb.train(params, train_set, valid_sets=[train_set, valid_set], early_stopping_rounds=5, verbose_eval=False)
    best_iteration = model.best_iteration
    feature_importances = model.feature_importance()

    return model, best_iteration, feature_importances

# Función de eliminación recursiva de características
def recursive_feature_elimination(X, y, params, threshold=0.05):
    remaining_features = list(X.columns)
    feature_importance_history = []

    while len(remaining_features) > 1:
        model, best_iteration, feature_importances = train_lgb(X[remaining_features], y, params)
        feature_importance_history.append((remaining_features.copy(), feature_importances))

        sorted_indices = np.argsort(feature_importances)
        num_features_to_drop = max(1, int(len(remaining_features) * threshold))
        features_to_drop = [remaining_features[i] for i in sorted_indices[:num_features_to_drop]]

        for feature in features_to_drop:
            remaining_features.remove(feature)

        print(f"Features dropped: {features_to_drop}")
        print(f"Remaining features: {len(remaining_features)}")

    return feature_importance_history, remaining_features

params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Ejemplo de uso de la eliminación recursiva de características
feature_importance_history, final_features = recursive_feature_elimination(X_train, y_train, params)

print(f"Final selected features: {final_features}")

# Modelado


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

def mean_target_encoding(train_df, test_df, target_col, cat_cols, n_splits=5):
    # Create a copy of the dataframes to avoid modifying the original ones
    train_df = train_df.copy()
    test_df = test_df.copy()

    # Initialize the KFold object
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Create a dictionary to hold the mean target values for each categorical column
    mean_encoded_cols = {col: f'{col}_mean_encoded' for col in cat_cols}

    # Add new columns for mean target encoding to the train and test dataframes
    for col in cat_cols:
        train_df[mean_encoded_cols[col]] = np.nan
        test_df[mean_encoded_cols[col]] = 0

    # Perform mean target encoding using KFold cross-validation
    for train_index, val_index in kf.split(train_df):
        X_train, X_val = train_df.iloc[train_index], train_df.iloc[val_index]

        for col in cat_cols:
            means = X_train.groupby(col)[target_col].mean()
            train_df.loc[val_index, mean_encoded_cols[col]] = X_val[col].map(means)

    # Calculate the overall mean for each categorical column and apply it to the test set
    for col in cat_cols:
        overall_means = train_df.groupby(col)[target_col].mean()
        test_df[mean_encoded_cols[col]] = test_df[col].map(overall_means)

        # For any missing values in the mean encoded columns, fill with the overall mean of the target
        train_df[mean_encoded_cols[col]].fillna(train_df[target_col].mean(), inplace=True)
        test_df[mean_encoded_cols[col]].fillna(train_df[target_col].mean(), inplace=True)

    # Drop the original categorical columns
    #train_df.drop(columns=cat_cols, inplace=True)
    #test_df.drop(columns=cat_cols, inplace=True)

    return train_df, test_df

In [ ]:
# Example usage with your data
cat_cols = X_train.select_dtypes(include=['object']).columns
target_col = 'ClosedWonOpportunity'

X_train, X_test = mean_target_encoding(pd.concat([X_train, y_train], axis=1), X_test, target_col, cat_cols)

print(X_train.shape)
print(X_test.shape)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.model_selection import StratifiedKFold
from category_encoders import TargetEncoder
import lightgbm as lgb


# Define the model with parameter flexibility
def get_model(n_estimators=100, random_state=42):
    return lgb.LGBMClassifier(n_estimators=n_estimators, random_state=random_state)

# Function to apply log transformation to skewed numerical features
def log_transform(x):
    return np.log1p(x)

# Prepare the training and testing datasets
# Assuming X_train and X_test are already defined DataFrames and y_train is the target Series

# Identify the identifier column
identifier_col = 'OpportunityID'

# Encode categorical variables
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object' and col != identifier_col]
numerical_cols = [col for col in X_train.columns if X_train[col].dtype != 'object' and col != identifier_col]

# One-Hot Encoding for low cardinality
low_cardinality_cols = [col for col in categorical_cols if X_train[col].nunique() < 10]
high_cardinality_cols = list(set(categorical_cols) - set(low_cardinality_cols))

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('log_transform', FunctionTransformer(log_transform, validate=False)),
    ('scaler', StandardScaler())
])

# Preprocessing for low cardinality categorical data
low_cardinality_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for high cardinality categorical data
high_cardinality_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('target', TargetEncoder())
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('low_cat', low_cardinality_transformer, low_cardinality_cols),
        ('high_cat', high_cardinality_transformer, high_cardinality_cols)
    ])

# Define the model
model = get_model()

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
custom_scores = []

print("Starting 5-fold cross-validation...")

for fold, (train_index, valid_index) in enumerate(kf.split(X_train, y_train), 1):
    print(f"Processing fold {fold}...")

    # Selecting features excluding the identifier column
    X_train_split = X_train.iloc[train_index].drop(columns=[identifier_col])
    X_valid_split = X_train.iloc[valid_index].drop(columns=[identifier_col])
    y_train_split = y_train.iloc[train_index]
    y_valid_split = y_train.iloc[valid_index]

    # Fit model
    clf.fit(X_train_split, y_train_split)
    print(f"Fold {fold}: Model training completed.")

    # Predict on validation set
    y_pred = clf.predict_proba(X_valid_split)[:, 1]
    y_valid_df = pd.DataFrame({'ClosedWonOpportunity': y_valid_split})
    y_pred_df = pd.DataFrame({'ClosedWonOpportunity': y_pred})

    # Calculate custom metric
    score = custom_metric(y_valid_df, y_pred_df)
    custom_scores.append(score)
    print(f"Calculating score for fold {fold}, score: {score}")

# Calculate and print the average custom metric score across the 5 folds
average_custom_score = np.mean(custom_scores)
print(f'Average Custom Metric Score across 5 folds: {average_custom_score:.3f}')

# Fit the final model on the entire training set excluding the identifier column
clf.fit(X_train.drop(columns=[identifier_col]), y_train)

# Make predictions on the test set excluding the identifier column
y_test_pred = clf.predict_proba(X_test.drop(columns=[identifier_col]))[:, 1]

# Hyper parameter tuning

In [ ]:
import optuna
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
from category_encoders import TargetEncoder
from sklearn.metrics import roc_auc_score, roc_curve, r2_score
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# Define log transform function
def log_transform(x):
    return np.log1p(x)

# Custom metric function
def custom_metric(solution: pd.DataFrame, submission: pd.DataFrame,
                  solution_target_col_name='ClosedWonOpportunity', submission_pred_col_name='ClosedWonOpportunity'):
    # Initialize DataFrame for metrics
    target = 'target'
    pred = 'pred'
    metric = pd.DataFrame()
    metric[target] = solution[solution_target_col_name].reset_index(drop=True)
    metric[pred] = submission[submission_pred_col_name].reset_index(drop=True)

    # Calculate deciles and metrics
    decile_labels = [f'D{i:02}' for i in range(10, 0, -1)]  # From D10 to D01
    metric['decil'] = pd.qcut(metric[pred], q=10, labels=decile_labels)

    # Cumulative target by decile
    cumulative_target_by_decile = (metric.groupby('decil')[target].sum() / metric[target].sum()).sort_index(ascending=False).cumsum()

    # Decile counts and best decile score
    decile_counts = metric['decil'].value_counts().sort_index(ascending=True)
    total_target = metric[target].sum()
    best_decil_score = (decile_counts / total_target).mean().round(4)

    # Create the perfect curve
    perfect_curve = np.cumsum([best_decil_score] * len(decile_labels))
    perfect_curve[perfect_curve > 1] = 1  # Cap values at 1

    # Calculate AUC of the perfect model
    decile_labels_scaled = np.linspace(0, 1, len(decile_labels))
    PAUC_max = np.trapz(perfect_curve, decile_labels_scaled)

    # Calculate Prioritization AUC
    priorization_auc = np.trapz(cumulative_target_by_decile, decile_labels_scaled) / PAUC_max

    # ROC AUC and Calibration Curve R2
    roc_auc = roc_auc_score(metric[target], metric[pred])
    fpr, tpr, thresholds = roc_curve(metric[target], metric[pred])
    prob_true, prob_pred = calibration_curve(metric[target], metric[pred], n_bins=20, strategy='uniform')
    r2 = r2_score(np.linspace(0, 1, len(prob_true)), prob_true)

    # Final combined metric
    final_metric = 0.5 * priorization_auc + 0.3 * roc_auc + 0.2 * r2

    return final_metric

# Define objective function for Optuna
def objective(trial):
    # Hyperparameter space
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 500),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'num_boost_round': 100
    }

    # Split the data
    X_train_df, X_valid_df, y_train_df, y_valid_df = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

    # Preprocessing steps
    identifier_col = 'OpporunityID'  # Assuming there's an identifier column named 'ID'
    categorical_cols = [col for col in X_train_df.columns if X_train_df[col].dtype == 'object' and col != identifier_col]
    numerical_cols = [col for col in X_train_df.columns if X_train_df[col].dtype != 'object' and col != identifier_col]

    # One-Hot Encoding for low cardinality
    low_cardinality_cols = [col for col in categorical_cols if X_train_df[col].nunique() < 10]
    high_cardinality_cols = list(set(categorical_cols) - set(low_cardinality_cols))

    # Preprocessing for numerical data
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('log_transform', FunctionTransformer(log_transform, validate=False)),
        ('scaler', StandardScaler())
    ])

    # Preprocessing for low cardinality categorical data
    low_cardinality_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Preprocessing for high cardinality categorical data
    high_cardinality_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('target', TargetEncoder())
    ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('low_cat', low_cardinality_transformer, low_cardinality_cols),
            ('high_cat', high_cardinality_transformer, high_cardinality_cols)
        ])

    # Define the model
    model = lgb.LGBMClassifier(**param)

    # Bundle preprocessing and modeling code in a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', model)
                         ])

    # Fit model
    clf.fit(X_train_df, y_train_df)

    # Make predictions
    y_pred = clf.predict_proba(X_valid_df)[:, 1]

    # Calculate custom metric
    solution = pd.DataFrame({'ClosedWonOpportunity': y_valid_df})
    submission = pd.DataFrame({'ClosedWonOpportunity': y_pred})
    custom_metric_score = custom_metric(solution, submission)

    return custom_metric_score


# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Print best trial
print("Best trial:")
trial = study.best_trial
print(trial.values)
print(trial.params)